How to use this hubert model.

declaimer
- This model is on a training process.
- The performance of the model will be posted after train.
- Since this model is trained on phoneme - such as ㄱ,ㄴ,ㄷ,ㅏ,ㅣ,ㅜ, etc -, unlike other models trained on grapheme - such as 가,나,다, etc-, it would less perform on small size dataset such as this zeroth-Korean dataset (about 50 hours). However, phoneme-based models would perform better on large datasets(over 100 hours with data argument) (이문학 and 장준혁,2019).
- There could be a comparison between grapheme and phoneme-based models
- You would load your sound file through python libraries such as touchaudio, soundfile, and librosa. 

ref

이문학, & 장준혁. (2019). 문자소 기반의 한국어 음성인식. 한국음향학회지 제, 38(5), 601-606.

In [1]:
!git clone https://github.com/Rosicast/jamo_unicode_for_colab.git

Cloning into 'jamo_unicode_for_colab'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [2]:
import jamo_unicode_for_colab.unicode as unicode

In [3]:
!pip install transformers==4.17.0
!pip install datasets==1.18.3
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 59.9 MB/s 
     |████████████████████████████████| 880 kB 58.1 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 101 kB 6.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=cacacd459fd06170169768aeef9ec3d109de52a06cce08613749330f0c7d6d8e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 311 kB 5.0 MB/s 
     |█████

In [4]:
from transformers import HubertForCTC, Wav2Vec2Processor
from datasets import load_dataset, load_metric
import torch
from jiwer import wer, cer

In [5]:
processor = Wav2Vec2Processor.from_pretrained("rosicast/hubert-large-ll60k-korean-zeroth-jamo")

model = HubertForCTC.from_pretrained("rosicast/hubert-large-ll60k-korean-zeroth-jamo")

ds = load_dataset("kresnik/zeroth_korean", "clean")

Downloading:   0%|          | 0.00/257 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/910 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Downloading:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset zeroth_korean_asr downloaded and prepared to /root/.cache/huggingface/datasets/kresnik___zeroth_korean_asr/clean/1.0.1/f6cf96a53d5512525e3113bab8048d36ce268658d6e0c40d45f65dfa3f0bc343. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
test_ds = ds['test']

In [7]:
test_ds

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 457
})

In [8]:
input_values = processor(test_ds[0]["audio"]["array"], return_tensors="pt").input_values  # Batch size 1
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])


/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [9]:
transcription

'ㄱㅗㅇㅈㅓㅇㅇㅟㄴㅡㄴ ㅇㅣㅊㅓㄴ ㅅㅣㅂ ㅅㅏㅁ ㄴㅕㄴ ㅇㅠㅌㅗㅇㅅㅏㄴㅇㅓㅂㅂㅏㄹㅈㅓㄴㅂㅓㅂ ㄱㅐㅈㅓㅇㅇㅡㄹ ㄱㅡㄴㄱㅓㄹㅗ ㅇㅣ ㄱㅗㅇㅇㅑㄱㅇㅣ ㅇㅣㅎㅐㅇㄷㅚㄴ ㄱㅓㅅㅇㅡㄹㅗ ㅍㅕㅇㄱㅏㅎㅐㅆㄷㅏ'

In [10]:
unicode.join_jamos(transcription)

'공정위는 이천 십 삼 년 유통산업발전법 개정을 근거로 이 공약이 이행된 것으로 평가했다'

In [11]:
test_ds[0]['text']

'공정위는 이천 십 삼 년 유통산업발전법 개정을 근거로 이 공약이 이행된 것으로 평가했다'